In [1]:
import gensim
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import nltk
import os
import re

import math

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data_folder = "data/ReligiousTexts/"
files = os.listdir(data_folder)

# Preprocessing

In [3]:
s = set(nltk.corpus.stopwords.words('english'))
s.add('')

In [4]:
texts = []
for file in files:
    with open(data_folder+file,encoding="ISO-8859-1") as f:
        text = f.read()
        text = re.split('[, \.\n]', text)
        text = [word.lower() for word in text if word.lower() not in s and word.isalpha() and len(word)>=3]
        texts.append(text)

In [5]:
common_dictionary = Dictionary(texts)

In [6]:
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

# Training

In [7]:
lda = gensim.models.LdaModel(common_corpus,id2word=common_dictionary ,num_topics=30)

In [8]:
lda.get_document_topics(common_dictionary.doc2bow(texts[1]))

[(22, 0.9999104)]

# Text v/s Topic

In [9]:
matrix = [[str(0) for i in range(150)] for i in range(len(texts))]

### Increase for more Radius
factor = 1
###

for i in range(len(texts)):
    topics = lda.get_document_topics(common_dictionary.doc2bow(texts[i]))
    for t in topics:
        matrix[i][t[0]] = str(t[1] * factor)

In [10]:
string = ','+','.join([str(i) for i in range(30)])
for i in range(len(texts)):
    string += '\nt' + str(i) + ',' + ','.join(matrix[i])

with open("text_topic_matrix.csv", "w") as f:
    f.write(string)

# Word v/s Topic

In [11]:
csv_string = 'word,value'

for t in lda.get_topic_terms(1, topn=100):
#     print( lda.id2word[t[0]],t[1],t[0])
    csv_string += '\n' + lda.id2word[t[0]] + ',' + str(t[1])
# t = lda.get_topic_terms(57, topn=100)

with open('topic1.csv', 'w') as f:
    f.write(csv_string)

MemoryError: 

In [ ]:
words = {}
for k,v in lda.id2word.items():
    words[v] = k
len(words)

In [ ]:
word_dict = {}
for topic in range(150):
    word_list = lda.get_topic_terms(topic, topn=1000)
    for word_tuple in word_list:
        word_id = word_tuple[0]
        word_score = word_tuple[1]
        if word_id not in word_dict:
            word_dict[word_id] = {}
        word_dict[word_id][topic] = word_score

In [ ]:
word_dict[words['interest']]